<a href="https://colab.research.google.com/github/3B032049/2025_ML_hws/blob/main/hw5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences


### 1. 將存好的模型和訓練權重讀回來用

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [5]:
from tensorflow.keras.models import model_from_json

In [6]:
model = model_from_json(open('imdb_model_architecture.json').read())
model.load_weights('imdb_model.weights.h5')

In [7]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 32)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 364,929 (1.39 MB)

 Trainable params: 364,929 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import pickle
f = open('tokenizer.pkl', 'rb')
tokens = pickle.load(f)
f.close()

In [10]:
from tensorflow.keras.preprocessing import sequence

In [45]:
re_tag = re.compile(r'<[^>]+>')

def rm_tags(text):
    return re_tag.sub('', text)

def predict_review(input_text):
    cleaned = rm_tags(input_text)
    seq = tokens.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=100)
    pred = model.predict(padded)[0][0]
    sentiment = "正面情感 😊" if pred > 0.5 else "負面情感 😠"
    return f"預測結果：{sentiment} (信心度：{pred:.4f})"


### 2. 測試模型

In [12]:
from tensorflow.keras.datasets.imdb import get_word_index

In [13]:
word_index = get_word_index()

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
word_index['this']

11

In [15]:
text = "this movie is worth seeing"

In [16]:
text.split()

['this', 'movie', 'is', 'worth', 'seeing']

In [17]:
seq = [word_index[x] for x in text.split()]

In [18]:
seq

[11, 17, 6, 287, 316]

In [19]:
seq = np.array(seq).reshape(1, len(seq))

### 3. 輸入我們自己的評論

In [29]:
input_text = "could of been so much better if properly cast directed and a better script"

In [30]:
input_seq = tokens.texts_to_sequences([input_text])

In [31]:
len(input_seq[0] )


14

In [32]:
pad_input_seq  = sequence.pad_sequences(input_seq , maxlen=100)

In [33]:
len(pad_input_seq[0])

100

In [34]:
pad_input_seq

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   96,    4,
          73,   34,   72,  124,   44, 2885,  173,  522,    2,    3,  124,
         225]], dtype=int32)

### 4. 輸入影評(數字串，長度為100)進行情意預測


In [35]:
predict_result=model.predict(pad_input_seq)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


In [36]:
predict_result

array([[0.00087554]], dtype=float32)

In [37]:
predict_result[0][0]


np.float32(0.00087553775)

In [38]:
SentimentDict={1:'正面的',0:'負面的'}


In [39]:
SentimentDict[round(predict_result[0][0])]


'負面的'

In [46]:
review_1 = "This movie was absolutely fantastic, with a powerful story and great performances."
url_1 = "https://www.imdb.com/review/rw2281110/"

review_2 = "This was a total waste of time. The plot made no sense and the acting was terrible."
url_2 = "https://www.imdb.com/review/rw5903593/"

review_3 = "The cinematography was stunning, but the pacing was a bit slow for my taste."
url_3 = "https://www.imdb.com/review/rw4470832/"

print("Review 1:", predict_review(review_1))
print("URL:", url_1)
print("\nReview 2:", predict_review(review_2))
print("URL:", url_2)
print("\nReview 3:", predict_review(review_3))
print("URL:", url_3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Review 1: 預測結果：正面情感 😊 (信心度：0.9910)
URL: https://www.imdb.com/review/rw2281110/
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

Review 2: 預測結果：負面情感 😠 (信心度：0.0000)
URL: https://www.imdb.com/review/rw5903593/
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

Review 3: 預測結果：負面情感 😠 (信心度：0.0973)
URL: https://www.imdb.com/review/rw4470832/


### 5. 用 Gradio 打造情意分析 Web App

In [23]:
# 在自己的電腦上就不需要再安裝一次了
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00


In [24]:
import gradio as gr

In [40]:
def imdb_score(input_text):
    seq = [word_index[x] for x in text.split()]
    seq = np.array(seq).reshape(1, len(seq))
    score = model.predict(seq)[0][0]
    return score

In [47]:
iface = gr.Interface(
    fn=predict_review,
    inputs=gr.Textbox(lines=6, placeholder="請輸入一段 IMDb 影評文字..."),
    outputs="text",
    title="IMDb 影評情意分析",
    description="輸入一段英文影評，我們會預測其為正面或負面情緒"
)

In [48]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e12959ec9cc66ffa03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [28]:
# 在自己的電腦上記得要設定share=True，這樣別人才看得到
# iface.launch(share=True)